In [ ]:
!pip install fasttext
!pip install numpy==1.26.4

In [ ]:
import re
import fasttext
from sklearn.model_selection import train_test_split
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import time

In [ ]:
# Danh sách stopwords tiếng Việt
with open('lib/vietnamese-stopwords.txt', 'r', encoding='utf-8') as file:
    stopwords = file.readlines()
stopwords = [word.replace('\n', '') for word in stopwords]
print(stopwords)

# Hàm tiền xử lý văn bản
def preprocess_text(text):
    text = text.lower()  # Chuyển về chữ thường
    text = re.sub(r'\d+', '', text)  # Loại bỏ chữ số
    text = re.sub(r'\s+', ' ', text)  # Loại bỏ khoảng trắng thừa
    text = re.sub(r'[^\w\s]', '', text)  # Loại bỏ dấu câu
    # Loại bỏ stopwords
    text = ' '.join([word for word in text.split() if word not in stopwords])
    return text

# Đọc dữ liệu từ file txt
def read_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    return [line.strip() for line in lines]


['a lô', 'a ha', 'ai', 'ai ai', 'ai nấy', 'ai đó', 'alô', 'amen', 'anh', 'anh ấy', 'ba', 'ba ba', 'ba bản', 'ba cùng', 'ba họ', 'ba ngày', 'ba ngôi', 'ba tăng', 'bao giờ', 'bao lâu', 'bao nhiêu', 'bao nả', 'bay biến', 'biết', 'biết bao', 'biết bao nhiêu', 'biết chắc', 'biết chừng nào', 'biết mình', 'biết mấy', 'biết thế', 'biết trước', 'biết việc', 'biết đâu', 'biết đâu chừng', 'biết đâu đấy', 'biết được', 'buổi', 'buổi làm', 'buổi mới', 'buổi ngày', 'buổi sớm', 'bà', 'bà ấy', 'bài', 'bài bác', 'bài bỏ', 'bài cái', 'bác', 'bán', 'bán cấp', 'bán dạ', 'bán thế', 'bây bẩy', 'bây chừ', 'bây giờ', 'bây nhiêu', 'bèn', 'béng', 'bên', 'bên bị', 'bên có', 'bên cạnh', 'bông', 'bước', 'bước khỏi', 'bước tới', 'bước đi', 'bạn', 'bản', 'bản bộ', 'bản riêng', 'bản thân', 'bản ý', 'bất chợt', 'bất cứ', 'bất giác', 'bất kì', 'bất kể', 'bất kỳ', 'bất luận', 'bất ngờ', 'bất nhược', 'bất quá', 'bất quá chỉ', 'bất thình lình', 'bất tử', 'bất đồ', 'bấy', 'bấy chầy', 'bấy chừ', 'bấy giờ', 'bấy lâu', 'bấy lâ

In [ ]:
# Tải và tiền xử lý bộ dữ liệu
df = pd.read_csv('content/dataset.csv', encoding='utf-8', sep=';')


print(df)
# Tiền xử lý dữ liệu
processed_data =[]
# for line in data:
#     parts = line.split(' ', 1)
#     if len(parts) == 2:
#         label, text = parts
#         processed_text = preprocess_text(text)
      #  processed_data.append(f"{label} {processed_text}")

for index, row in df.iterrows():
    # Lấy nhãn từ cột 'category', giả sử nhãn là một chuỗi với nhiều từ
    category = row['label']  # Đây là nhãn có thể chứa nhiều từ (ví dụ: "Acute Hepatopancreatic Necrosis")

    # Chuyển nhãn thành định dạng "__label__" và nối các từ trong nhãn bằng dấu gạch dưới
    label = f"__label__{category.replace(' ', '_')}"  # Nối các từ bằng dấu gạch dưới

    text = row['text']  # Lấy văn bản từ cột 'headline'

    # Tiền xử lý văn bản
    processed_text = preprocess_text(text)

    # Ghép lại nhãn và văn bản đã xử lý
    processed_data.append(f"{label} {processed_text}")

print(processed_data)  # Kiểm tra số lượng mẫu và cột



                                                   text             label
0     BỘ GIÁO DỤC VÀ ĐÀO TẠO CỘNG HÒA XÃ HỘI CHỦ NGH...  __label__Dao_tao
1     sang học một chương trình (toàn bộ những dự ki...  __label__Dao_tao
2     của bộ trưởng bộ giáo dục (hoạt động nhằm tác ...  __label__Dao_tao
3     hành (cây thân ngầm, hình dẹp, mang nhiều lá m...  __label__Dao_tao
4     về sau như sau  đối tượng (người, vật, hiện tư...  __label__Dao_tao
...                                                 ...               ...
3570  Để Cần Thơ phát triển, cần có nếp sống văn min...     __label__Khac
3571  Cần Thơ cần có một nếp sống văn minh, với ý th...     __label__Khac
3572  Để đảm bảo Cần Thơ phát triển bền vững, cần có...     __label__Khac
3573  Cần Thơ phải có nếp sống văn minh, với ý thức ...     __label__Khac
3574  Nếp sống văn minh là điều cần thiết ở Cần Thơ,...     __label__Khac

[3575 rows x 2 columns]
['__label____label__Dao_tao giáo dục đào cộng hòa xã hội chủ nghĩa việt nam trường đại 

In [ ]:
print(len(processed_data))  # Kiểm tra độ dài danh sách
print(processed_data[:5])   # Kiểm tra một vài phần tử đầu tiên


3575
['__label____label__Dao_tao giáo dục đào cộng hòa xã hội chủ nghĩa việt nam trường đại học càn thơ độc lập hạnh phúc trạng thái sung sướng cảm hoàn toàn nguyện hạnh phúc trẻ thơ hưởng hạnh phúc vẻ mặt tràn trề hạnh phúc màn trình diễn nghệ thuật sô quảng cáo sô trình diễn thời trang sô diễn chạy sô biểu diễn liên tiếp hdđhct thơ hiếm rau ráu dụng cụ leo tre gỗ hai song song nối liền ngang ngắn thành bậc bắc thang nấc thang đời b thông tục đàn đanh đá lắm thô bỉ tiếng chửi tiếng mắng nặc nô hướng học chương sách công trình khoa học nội dung tương đối trọn vẹn quyển sách chương luận án chương', '__label____label__Dao_tao học chương trình toàn dự kiến hoạt động trình định thời gian định nêu vắn tắt chương trình ca nhạc chương trình điện khí hoá miền núi ngành cản trở hoạt động ngáng trở công đào đối sinh viên tổng thể khoa học giới hữu sinh trình sống khóa trở diệt vi trùng thuốc sát trùng sát trùng vết thương cẩn thận trường căng phình hút cơm trương bụng trương to trương phềnh phền

In [ ]:
# Chia dữ liệu thành tập huấn luyện và kiểm tra
train_data, test_data = train_test_split(processed_data, test_size=0.2, random_state=42)

# Lưu dữ liệu đã tiền xử lý vào các file
os.makedirs('/content/data_fasstext', exist_ok=True)
train_data_path = '/content/data_fasstext/train_data.txt'
test_data_path = '/content/data_fasstext/test_data.txt'

with open(train_data_path, 'w', encoding='utf-8') as f:
    f.write('\n'.join(train_data))

with open(test_data_path, 'w', encoding='utf-8') as f:
    f.write('\n'.join(test_data))


In [ ]:
print(test_data)

['__label____label__KTX sinh viên ưu đãi tương khoản nội quy sinh viên gửi phương tiện phương tiện trở đi sinh viên nộp phí vãng lai xe đảm xe sinh viên', '__label____label__Hoc_tap_ren_luyen kết rèn luyện sinh viên bậc đại học hình thức quy trường đại học thơ giá quyết định hiệu lực học kỳ học thay quyết định qđđhct', '__label____label__Dao_tao trình đào chương trình tương nựng khái quát mẹ nựng nịu nựng nịu vỗ mãi đứa bé ngủ thông kết thu công trình tiến triển vật kết học tập kết đánh giá kiểm định chất chương trình đào mục tiêu chỗ bắn trúng mục tiêu phát hiện mục tiêu đào chương trình đào chuẩn đầu kết hoạt động sản xuất kinh doanh phân biệt đầu dự trưởng đầu', '__label____label__KTX tham gia đợt tổng vệ sinh ktx trung tâm pvsv tổ chức thiết khu vực ktx lưu bát hương thờ cúng phòng khu vực ktx nuôi cá vật nuôi phòng khu vực', '__label____label__Dao_tao viên ctđtclc ctđttt thực hiện nhiệm vụ đối học quy định luật giáo dục đại học trường đhct quyền sinh viên quyền sinh viên quy định 

In [ ]:
# Train mô hình
start_time = time.time()
model = fasttext.train_supervised(input=train_data_path, epoch=100, lr=1.0, wordNgrams=3, verbose=2, minCount=1, dim=100)
end_time = time.time()
training_time = end_time - start_time
print(f"Training time: {training_time} seconds")

# Lưu mô hình
model.save_model('/content/data_fasstext/fasttext_model_5.bin')

Training time: 10.588306903839111 seconds


In [ ]:
import time
import fasttext
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

In [ ]:
train_data_path = '/content/data_fasstext/train_data.txt'
test_data_path = '/content/data_fasstext/test_data.txt'


In [ ]:
test_data  = pd.read_csv(test_data_path)
print(test_data.head())  # Hiển thị 5 dòng đầu của dữ liệu
print(test_data.columns)  # Hiển thị tên các cột


  __label____label__KTX sinh viên ưu đãi tương khoản nội quy sinh viên gửi phương tiện phương tiện trở đi sinh viên nộp phí vãng lai xe đảm xe sinh viên
0  __label____label__Hoc_tap_ren_luyen kết rèn lu...                                                                                                    
1  __label____label__Dao_tao trình đào chương trì...                                                                                                    
2  __label____label__KTX tham gia đợt tổng vệ sin...                                                                                                    
3  __label____label__Dao_tao viên ctđtclc ctđttt ...                                                                                                    
4  __label____label__Tot_nghiep xét nghiệp sinh v...                                                                                                    
Index(['__label____label__KTX sinh viên ưu đãi tương khoản nội quy sinh viên gửi p

In [ ]:
true_labels = []
headlines = []

with open(test_data_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()  # Loại bỏ khoảng trắng thừa
            if not line:
                continue  # Bỏ qua dòng trống

            # Tách nhãn và dữ liệu
            label, text = line.split(' ', 1)  # Tách tại khoảng trắng đầu tiên
            true_labels.append(label)
            headlines.append(text)



In [ ]:
print(true_labels)
print(headlines)

['__label____label__KTX', '__label____label__Hoc_tap_ren_luyen', '__label____label__Dao_tao', '__label____label__KTX', '__label____label__Dao_tao', '__label____label__Tot_nghiep', '__label____label__Hoc_tap_ren_luyen', '__label____label__Hoc_tap_ren_luyen', '__label____label__Khac', '__label____label__Tot_nghiep', '__label____label__Hoc_tap_ren_luyen', '__label____label__Khac', '__label____label__Hoc_tap_ren_luyen', '__label____label__Dao_tao', '__label____label__Khac', '__label____label__Khac', '__label____label__Khen_thuong_ky_luat', '__label____label__KTX', '__label____label__Dao_tao', '__label____label__KTX', '__label____label__Khen_thuong_ky_luat', '__label____label__KTX', '__label____label__Khen_thuong_ky_luat', '__label____label__KTX', '__label____label__Hoc_tap_ren_luyen', '__label____label__Hoc_tap_ren_luyen', '__label____label__Tot_nghiep', '__label____label__Hoc_tap_ren_luyen', '__label____label__Hoc_tap_ren_luyen', '__label____label__Dao_tao', '__label____label__Dao_tao', '

In [ ]:

# Hàm để load dữ liệu và test mô hình
def evaluate_model(model, test_data_path):
    true_labels = []
    headlines = []

    with open(test_data_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()  # Loại bỏ khoảng trắng thừa
            if not line:
                continue  # Bỏ qua dòng trống

            # Tách nhãn và dữ liệu
            label, text = line.split(' ', 1)  # Tách tại khoảng trắng đầu tiên
            true_labels.append(label)
            headlines.append(text)

    # Dự đoán nhãn
    pred_labels = [model.predict(text)[0][0] for text in headlines]
    # Tính toán các chỉ số
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, pred_labels, average='weighted')
    accuracy = accuracy_score(true_labels, pred_labels)

    return accuracy, precision, recall, f1

# Khởi tạo các danh sách lưu kết quả
accuracies, precisions, recalls, f1s, train_times, test_times = [], [], [], [], [], []

# Thực hiện 10 lần train và test
for i in range(10):
    print(f"Iteration {i+1}/10")

    # Train mô hình
    start_time = time.time()
    model = fasttext.train_supervised(input=train_data_path, epoch=100, lr=1.0, wordNgrams=3, verbose=2, minCount=1, dim=100)
    end_time = time.time()
    train_time = end_time - start_time
    train_times.append(train_time)
    print(f"Training time: {train_time} seconds")

    # Test mô hình
    test_start_time = time.time()
    accuracy, precision, recall, f1 = evaluate_model(model, test_data_path)
    test_end_time = time.time()
    test_time = test_end_time - test_start_time
    test_times.append(test_time)

    print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1: {f1}, Test time: {test_time} seconds")

    # Lưu kết quả
    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1s.append(f1)

# Tính trung bình
print("\nAverage Results over 10 runs:")
print(f"Average Accuracy: {np.mean(accuracies):.4f}")
print(f"Average Precision: {np.mean(precisions):.4f}")
print(f"Average Recall: {np.mean(recalls):.4f}")
print(f"Average F1 Score: {np.mean(f1s):.4f}")
print(f"Average Training Time: {np.mean(train_times):.4f} seconds")
print(f"Average Testing Time: {np.mean(test_times):.4f} seconds")


Iteration 1/10
Training time: 9.831099510192871 seconds
Accuracy: 0.9706293706293706, Precision: 0.9709337077104956, Recall: 0.9706293706293706, F1: 0.970291494283238, Test time: 0.05262875556945801 seconds
Iteration 2/10
Training time: 9.880964517593384 seconds
Accuracy: 0.9706293706293706, Precision: 0.9709337077104956, Recall: 0.9706293706293706, F1: 0.970291494283238, Test time: 0.03837275505065918 seconds
Iteration 3/10
Training time: 10.386171102523804 seconds
Accuracy: 0.9706293706293706, Precision: 0.9709337077104956, Recall: 0.9706293706293706, F1: 0.970291494283238, Test time: 0.036681175231933594 seconds
Iteration 4/10
Training time: 10.858741521835327 seconds
Accuracy: 0.9706293706293706, Precision: 0.9709337077104956, Recall: 0.9706293706293706, F1: 0.970291494283238, Test time: 0.03586697578430176 seconds
Iteration 5/10
Training time: 10.0751793384552 seconds
Accuracy: 0.9706293706293706, Precision: 0.9709337077104956, Recall: 0.9706293706293706, F1: 0.970291494283238, Te